In [ ]:
import pandas as pd
import numpy as np
import math

## EDA

In [2]:
fraud_df = pd.read_parquet('data/transaction_fraud_data.parquet')
currency_df = pd.read_parquet('data/historical_currency_exchange.parquet')

In [3]:
print(fraud_df.columns)
fraud_df.head()

Index(['transaction_id', 'customer_id', 'card_number', 'timestamp',
       'vendor_category', 'vendor_type', 'vendor', 'amount', 'currency',
       'country', 'city', 'city_size', 'card_type', 'is_card_present',
       'device', 'channel', 'device_fingerprint', 'ip_address',
       'is_outside_home_country', 'is_high_risk_vendor', 'is_weekend',
       'last_hour_activity', 'is_fraud'],
      dtype='object')


,transaction_id,customer_id,card_number,timestamp,vendor_category,vendor_type,vendor,amount,currency,country,...,is_card_present,device,channel,device_fingerprint,ip_address,is_outside_home_country,is_high_risk_vendor,is_weekend,last_hour_activity,is_fraud
0,TX_a0ad2a2a,CUST_72886,6646734767813109,2024-09-30 00:00:01.034820,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,...,False,iOS App,mobile,e8e6160445c935fd0001501e4cbac8bc,197.153.60.199,False,False,False,"{'num_transactions': 1197, 'total_amount': 334...",False
1,TX_3599c101,CUST_70474,376800864692727,2024-09-30 00:00:01.764464,Entertainment,gaming,Steam,3368.97,BRL,Brazil,...,False,Edge,web,a73043a57091e775af37f252b3a32af9,208.123.221.203,True,True,False,"{'num_transactions': 509, 'total_amount': 2011...",True
2,TX_a9461c6d,CUST_10715,5251909460951913,2024-09-30 00:00:02.273762,Grocery,physical,Whole Foods,102582.38,JPY,Japan,...,False,Firefox,web,218864e94ceaa41577d216b149722261,10.194.159.204,False,False,False,"{'num_transactions': 332, 'total_amount': 3916...",False
3,TX_7be21fc4,CUST_16193,376079286931183,2024-09-30 00:00:02.297466,Gas,major,Exxon,630.60,AUD,Australia,...,False,iOS App,mobile,70423fa3a1e74d01203cf93b51b9631d,17.230.177.225,False,False,False,"{'num_transactions': 764, 'total_amount': 2201...",False
4,TX_150f490b,CUST_87572,6172948052178810,2024-09-30 00:00:02.544063,Healthcare,medical,Medical Center,724949.27,NGN,Nigeria,...,False,Chrome,web,9880776c7b6038f2af86bd4e18a1b1a4,136.241.219.151,True,False,False,"{'num_transactions': 218, 'total_amount': 4827...",True


In [21]:
print(currency_df.columns)
currency_df.head()

Index(['date', 'AUD', 'BRL', 'CAD', 'EUR', 'GBP', 'JPY', 'MXN', 'NGN', 'RUB',
       'SGD', 'USD'],
      dtype='object')


,date,AUD,BRL,CAD,EUR,GBP,JPY,MXN,NGN,RUB,SGD,USD
0,2024-09-30,1.443654,5.434649,1.351196,0.895591,0.747153,142.573268,19.694724,1668.736400,94.133735,1.280156,1
1,2024-10-01,1.442917,5.444170,1.352168,0.897557,0.746956,143.831429,19.667561,1670.694524,92.898519,1.284352,1
2,2024-10-02,1.449505,5.425444,1.348063,0.903056,0.752241,143.806861,19.606748,1669.653006,94.583198,1.286983,1
3,2024-10-03,1.456279,5.442044,1.351451,0.906018,0.754584,146.916773,19.457701,1670.097873,95.655442,1.294391,1
4,2024-10-04,1.460930,5.477788,1.355260,0.906452,0.761891,146.592323,19.363467,1649.763738,94.755337,1.296800,1


In [ ]:
fraud_ratio = round(fraud_df['is_fraud'].mean(), 1)

print(f"fraud_ratio: {fraud_ratio}")

fraud_ratio: 0.2


In [5]:
frauds = fraud_df[fraud_df['is_fraud']]
top5 = frauds['country'].value_counts().head(5)

print("top countries by fraud activity: ", ','.join(top5.index))

top countries by fraud activity:  Russia,Mexico,Brazil,Nigeria,Australia


In [ ]:
fraud_df['timestamp'] = pd.to_datetime(fraud_df['timestamp'])
fraud_df['hour'] = fraud_df['timestamp'].dt.floor('h')
tx_per_client_hour = fraud_df.groupby(['customer_id', 'hour']).size()
mean_tx_per_hour = tx_per_client_hour.mean()

print("avg number of transaction in one hour across clients: ", np.floor(mean_tx_per_hour * 10) / 10)

avg number of transaction in one hour across clients:  2.4


In [32]:
high_risk = fraud_df[fraud_df['is_high_risk_vendor']]
fraud_ratio_high_risk = high_risk['is_fraud'].mean()

print("Fraud ration among high risk transactions: ", np.ceil(fraud_ratio_high_risk * 10) / 10 )

Fraud ration among high risk transactions:  0.2


In [8]:
df_real_cities = fraud_df[fraud_df['city'] != 'Unknown City']
city_avg = df_real_cities.groupby('city')['amount'].mean()

print(city_avg.idxmax())

New York


In [9]:
fastfood_df = fraud_df[fraud_df['vendor_type'] == 'fast_food']
fastfood_df_real = fastfood_df[fastfood_df['city'] != 'Unknown City']
mean_amount_by_city = fastfood_df_real.groupby('city')['amount'].mean()

print("fast foodiest city: ", mean_amount_by_city.idxmax())

fast foodiest city:  Chicago


In [ ]:
def merge_currency_and_fraud(df, currency_df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['timestamp']).dt.date
    currency_df['date'] = pd.to_datetime(currency_df['date']).dt.date
    
    merged = df.merge(currency_df, on='date', how='left')
    merged['rate_to_usd'] = merged.apply(lambda row: row[row['currency']], axis=1)
    merged['usd_amount'] = merged['amount'] / merged['rate_to_usd']
    
    return merged


merged_legit = merge_currency_and_fraud(fraud_df[~fraud_df['is_fraud']], currency_df)
merged_non_legit = merge_currency_and_fraud(fraud_df[fraud_df['is_fraud']], currency_df)

In [ ]:
mean_legit = merged_legit['usd_amount'].mean()
std_legit = merged_legit['usd_amount'].std()
mean_non_legit = merged_non_legit['usd_amount'].mean()
std_non_legit = merged_non_legit['usd_amount'].std()

print("mean for legit transactions:", math.ceil(mean_legit))
print("std for legit transactions:", math.ceil(std_legit))
print("mean for non-legit transactions:", math.ceil(mean_non_legit))
print("std for non-legit transactions:", math.ceil(std_non_legit))

mean for legit transactions: 460
std for legit transactions: 418
mean for non-legit transactions: 875
std for non-legit transactions: 1350


In [ ]:
fraud_df['unique_merchants_last_hour'] = fraud_df['last_hour_activity'].apply(
    lambda x: x['unique_merchants'] if pd.notnull(x) else None)

median_activities = fraud_df.groupby('customer_id')['unique_merchants_last_hour'].median()
q95 = median_activities.quantile(0.95)
pontentially_dangerous_count = (median_activities > q95).sum()

print("number of pontentially dangerous clients:", pontentially_dangerous_count)


number of pontentially dangerous clients: 229


## EDA выводы:

- Доля мошеннических транзакций составляет 20% — это довольно высокая доля, требующая внимания.
- Топ-5 стран по активности мошенничества: Россия, Мексика, Бразилия, Нигерия, Австралия. Это географические направления повышенного риска.
- В среднем клиент совершает 2.4 транзакции в час — важный показатель клиентской активности для мониторинга.
- Доля мошенничества среди транзакций у продавцов с высоким уровнем риска также 20% — подтверждает целесообразность выделения и отдельного анализа high risk категорий.
- Город с наибольшей средней суммой транзакций — Нью-Йорк, что может указывать на концентрацию крупных операций.
- Город с наибольшей средней суммой для fast food операций — Чикаго — потенциально интересный сегмент для дальнейшего изучения.
- Средняя сумма легитимных транзакций — \$460, со стандартным отклонением \$418, тогда как средняя сумма мошеннических — \$875 с высоким стандартным отклонением \$1350. Значительная дисперсия у мошеннических операций указывает на нестабильность и разнообразие схем.


## Продуктовые гипотезы и рекомендации

- Большая доля мошенничества (20%) указывает на необходимость усиления системы выявления и предотвращения мошеннических транзакций, особенно для клиентов и продавцов из указанных стран.
- Рекомендуется создать специализированные правила или автоматизированные проверки для продавцов из списков high risk, поскольку риск мошенничества у них значительно выше.
- Клиенты с высокой частотой транзакций за час (среднее — 2.4) могут быть целевой группой для дополнительного мониторинга или ограничения по количеству операций в короткий период.
- Определение и мониторинг «потенциально опасных клиентов» (229 человек) может помочь в предотвращении мошеннической активности через персональные ограничения или усиление верификации.

## Технические гипотезы и направления

- Высокое стандартное отклонение мошеннических транзакций (1350 USD) говорит о том, что мошеннические операции имеют непредсказуемый размер; поэтому стоит включить в модели признаки, связанные с аномальной величиной транзакций.
- Значимые различия в поведении транзакций (например, доля мошенничества у high risk vendors) подразумевают необходимость сегментированного анализа и построения специализированных моделей для разных категорий продавцов.
- Использование признаков, связанных с географией (страны и города), позволит улучшить качество предсказаний мошенничества.
- Рассмотреть дополнительные каналы и устройства, участвующие в транзакциях, для выявления корреляций с мошенничеством.
- Внедрение реального времени мониторинга активности клиентов с высоким уровнем last_hour_activity и уникальных продавцов, чтобы оперативно выявлять подозрительные паттерны